In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
import pylab


# Загрузим данные из таблицы

In [2]:
# Загрузим первый датафрейм звонки
df_calls = pd.read_excel('/Users/leonid/Downloads/Задание.xlsx', sheet_name='call').drop([0]).drop('Unnamed: 5', axis=1)
# удалили строку, дублирующую название колонок на русском и колонку, поясняющую result_id 

In [3]:
df_calls.head()

,task_id,call_id,agent_name,call_dt,call_result_id,call_time_sec
1,843083337,6645916908#16711160876,sakungurceva,2020-03-01 00:00:00,6,298
2,843083337,6750206063#16711160876,sakungurceva,2020-03-17 00:00:00,6,9
3,843083337,6769109629#16711160876,sakungurceva,2020-03-24 00:00:00,6,111
4,843083337,6890964670#16711160876,sakungurceva,2020-04-15 00:00:00,5,7
5,843083337,6926247810#16711160876,lapleshivenkova,2020-04-15 00:00:00,3,1329


In [4]:
# Загрузим второй датафрейм задания
df_tasks = pd.read_excel('/Users/leonid/Downloads/Задание.xlsx', sheet_name='task').drop(0)

In [5]:
df_tasks.head()

,task_id,hid,finish_dt_task,base_id,task_result_id
1,843083337,36d2f49fc49f572677416d54536a41c6,2020-04-15 00:00:00,7553,6
2,844351919,ff34fad215457a0706186e347b6a3ac8,2020-04-27 00:00:00,7553,6
3,844406657,d9d9e1c6f5c95b3d8c72bb528bb06022,2020-04-10 00:00:00,7553,5
4,845152792,afdf70661693bbba8c31fb0b321b818c,2020-04-25 00:00:00,7553,5
5,845814859,9b3a7955c46a7e9658288cb8acc43eac,2020-04-05 00:00:00,7553,3


In [6]:
# Загрузим третий датафрейм - продукты
df_products = pd.read_excel('/Users/leonid/Downloads/Задание.xlsx', sheet_name='result_product').drop(0)

In [7]:
df_products.head()

,hid,product_type_cd,approve_dt,reject_dt,using_dt
1,0004c6142b4bf98fdb51ec0fead099b0,INV,2020-05-21 00:00:00,NaN,NaN
2,0008ddccf50e9a72ee9c6223fb45e4c4,INV,2020-05-29 00:00:00,NaN,NaN
3,000a2e5621a118118f42560c9821e6a1,INV,2020-04-27 00:00:00,NaN,2020-04-29 00:00:00
4,001a43fffa79a5b60c01473eeff9fd1f,INV,NaN,2020-04-01 00:00:00,NaN
5,001c27126f4fd13cc4126d4906c48425,INV,2020-05-06 00:00:00,NaN,NaN


# Предобработка данных

### Проверим на дубликаты

In [8]:
df_calls.duplicated().sum()

0

In [9]:
df_tasks.duplicated().sum()

0

In [10]:
df_products.duplicated().sum()

0

In [11]:
# дубликаты не найдены

### Проверим на пропуски

In [12]:
df_calls.isna().sum() 

task_id           0
call_id           0
agent_name        0
call_dt           0
call_result_id    0
call_time_sec     0
dtype: int64

In [13]:
df_tasks.isna().sum() 

task_id           0
hid               0
finish_dt_task    0
base_id           0
task_result_id    0
dtype: int64

In [14]:
df_products.isna().sum()
# судя по всему пропуски связаны с тем, то пользователь может совершить не все действия

hid                   0
product_type_cd       0
approve_dt         2275
reject_dt          3735
using_dt           4038
dtype: int64

In [15]:
df_products.shape

(5999, 5)

In [16]:
#  проверим, есть ли полностью пропущенные значения для действий клиента 
df_products[df_products[['approve_dt','reject_dt','using_dt']].isnull().all(axis = 1)]

,hid,product_type_cd,approve_dt,reject_dt,using_dt
384,0ee28a3b82fc23356e5f33240c324c29,INV,NaN,NaN,NaN
1455,3bc14fa6929915ee34cfc2035a27c334,INV,NaN,NaN,NaN
1792,4b22d259df1ebefb70d5056de40effb0,INV,NaN,NaN,NaN
2725,7434403d315dc0dd7582abfb49b98414,INV,NaN,NaN,NaN
2862,7a190118d423efc20e7fe5d0d3b9d469,INV,NaN,NaN,NaN
2927,7ccf7ea51c4e50472085cd20a659a030,INV,NaN,NaN,NaN
3021,804dbacd5e814d62628b967ab835076d,INV,NaN,NaN,NaN
3369,8f393a26a2405776ec5d41a891831ae4,INV,NaN,NaN,NaN
3457,930d5efa433377e081609ba6f041aaa5,INV,NaN,NaN,NaN
4426,bd93d551d74f64fdfccef583d69ba856,INV,NaN,NaN,NaN


 Пока непонятно как распряжаться с пропусками в таблице с действиями после продужи, так как мы не знаем специфику продукта. Предполагаю, что эти люди просто не совершили никаких действий. Держим это в голове, при дальшейшем соединении датафреймов, это не должно повлиять на ход исследования. 

### Провериям корректность считывания данных

### df_calls

In [17]:
df_calls.info()
# данные прочитаны не совсем корректно, заменим тип данных для колонок [task_id,call_dt,call_result_id,call_result_id,call_time_sec]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39598 entries, 1 to 39598
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   task_id         39598 non-null  object
 1   call_id         39598 non-null  object
 2   agent_name      39598 non-null  object
 3   call_dt         39598 non-null  object
 4   call_result_id  39598 non-null  object
 5   call_time_sec   39598 non-null  object
dtypes: object(6)
memory usage: 2.1+ MB


In [18]:
df_calls = df_calls.astype({'task_id' : 'int', 'call_dt' : 'datetime64', 'call_result_id': 'int','call_time_sec' : 'int' }, copy = True)
df_calls.info() #преобразовали успешно, пропусков нет

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39598 entries, 1 to 39598
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   task_id         39598 non-null  int64         
 1   call_id         39598 non-null  object        
 2   agent_name      39598 non-null  object        
 3   call_dt         39598 non-null  datetime64[ns]
 4   call_result_id  39598 non-null  int64         
 5   call_time_sec   39598 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 2.1+ MB


In [19]:
firts_date = df_calls['call_dt'].min()
last_date = df_calls['call_dt'].max()
print('Данные от %s до %s, разница в %s.' % (firts_date, last_date, last_date - firts_date))

Данные от 2020-03-01 00:00:00 до 2020-05-30 00:00:00, разница в 90 days 00:00:00.


In [20]:
# данные по звонкам представлены с 1 марта по 30 мая 2020 года

### df_tasks

In [21]:
df_tasks.info()
# аналогичная процедура для датафрейма df_tasks

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23119 entries, 1 to 23119
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   task_id         23119 non-null  object
 1   hid             23119 non-null  object
 2   finish_dt_task  23119 non-null  object
 3   base_id         23119 non-null  object
 4   task_result_id  23119 non-null  object
dtypes: object(5)
memory usage: 1.1+ MB


In [22]:
df_tasks = df_tasks.astype({'task_id':'int', 'finish_dt_task' : 'datetime64','base_id' : 'int' ,'task_result_id' : 'int' }, copy = True)
df_tasks.info() #преобразовали успешно, пропусков нет

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23119 entries, 1 to 23119
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   task_id         23119 non-null  int64         
 1   hid             23119 non-null  object        
 2   finish_dt_task  23119 non-null  datetime64[ns]
 3   base_id         23119 non-null  int64         
 4   task_result_id  23119 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 1.1+ MB


In [23]:
firts_date = df_tasks['finish_dt_task'].min()
last_date = df_tasks['finish_dt_task'].max()
print('Данные от %s до %s, разница в %s.' % (firts_date, last_date, last_date - firts_date))

Данные от 2020-04-01 00:00:00 до 2020-05-30 00:00:00, разница в 59 days 00:00:00.


In [24]:
# данные по заданиям представлены с 1 апреля по 30 мая 2020 года

### df_products

In [25]:
df_products.info()
# аналогичная процедура для датафрейма df_products

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5999 entries, 1 to 5999
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   hid              5999 non-null   object
 1   product_type_cd  5999 non-null   object
 2   approve_dt       3724 non-null   object
 3   reject_dt        2264 non-null   object
 4   using_dt         1961 non-null   object
dtypes: object(5)
memory usage: 281.2+ KB


In [26]:
df_products = df_products.astype({'approve_dt' : 'datetime64','reject_dt' : 'datetime64','using_dt' : 'datetime64', }, copy = True)
df_products.info() #преобразовали успешно, пропусков нет

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5999 entries, 1 to 5999
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   hid              5999 non-null   object        
 1   product_type_cd  5999 non-null   object        
 2   approve_dt       3724 non-null   datetime64[ns]
 3   reject_dt        2264 non-null   datetime64[ns]
 4   using_dt         1961 non-null   datetime64[ns]
dtypes: datetime64[ns](3), object(2)
memory usage: 281.2+ KB


In [27]:
firts_date = (df_products[['approve_dt','reject_dt','using_dt']].min()).min()
last_date = (df_products[['approve_dt','reject_dt','using_dt']].max()).max()
print(firts_date, last_date)
# print('Данные от %s до %s, разница в %s.' % (firts_date, last_date, last_date - firts_date))

2020-04-01 00:00:00 2020-06-09 00:00:00


# Задание 1

Составить общую таблицу с данными из всех источников и словарей.

In [28]:
# выгрузим словари и разобъем их на таблицы
df_dict = pd.read_excel('/Users/leonid/Downloads/Задание.xlsx', sheet_name='Словари', header = 1)

In [29]:
df_dict

,call_result_id,call_result_translation,Unnamed: 2,task_result_id,task_result_translation,Unnamed: 5,base_id,base_translation
0,6.0,"Дозвон, Отложить",NaN,6,Успешно,NaN,7553.0,База 1
1,5.0,Недозвон,NaN,5,Отказ,NaN,3145.0,База 2
2,3.0,"Дозвон, Успешно",NaN,3,Недозвон,NaN,3445.0,База 3
3,1.0,"Дозвон, Отказ",NaN,1,Дубль,NaN,7554.0,База 4
4,2.0,"Дозвон, Перезвонить",NaN,4,Неуспешно из-за другой заявки,NaN,NaN,NaN
5,4.0,Не было звонка,NaN,7,Черный список,NaN,NaN,NaN
6,NaN,NaN,NaN,2,Не было звонка,NaN,NaN,NaN


In [30]:
# создадим три отдельные таблицы, из них сделаем словри и позже применим функциию map для соответсвующих столбцов

In [31]:
call_result_dict = df_dict[['call_result_id','call_result_translation']].dropna().set_index('call_result_id')['call_result_translation'].to_dict()
task_result_dict = df_dict[['task_result_id','task_result_translation']].dropna().set_index('task_result_id')['task_result_translation'].to_dict()
base_result_dict = df_dict[['base_id','base_translation']].dropna().set_index('base_id')['base_translation'].to_dict()

In [32]:
call_result_dict

{6.0: 'Дозвон, Отложить',
 5.0: 'Недозвон',
 3.0: 'Дозвон, Успешно',
 1.0: 'Дозвон, Отказ',
 2.0: 'Дозвон, Перезвонить',
 4.0: 'Не было звонка'}

In [33]:
df_calls.call_result_id = df_calls.call_result_id.map(call_result_dict)
df_tasks.task_result_id = df_tasks.task_result_id.map(task_result_dict)
df_tasks.base_id = df_tasks.base_id.map(base_result_dict)

In [34]:
# создиним три таблицы

In [66]:
df = df_calls.merge(df_tasks).merge(df_products, how = 'left').rename(columns = {'call_result_id' : 'call_result_translation', 'task_result_id' : 'task_result_translation' , 'base_id' : 'base_translation'})




In [67]:
df

,task_id,call_id,agent_name,call_dt,call_result_translation,call_time_sec,hid,finish_dt_task,base_translation,task_result_translation,product_type_cd,approve_dt,reject_dt,using_dt
0,843083337,6645916908#16711160876,sakungurceva,2020-03-01,"Дозвон, Отложить",298,36d2f49fc49f572677416d54536a41c6,2020-04-15,База 1,Успешно,INV,NaT,2020-04-22,NaT
1,843083337,6750206063#16711160876,sakungurceva,2020-03-17,"Дозвон, Отложить",9,36d2f49fc49f572677416d54536a41c6,2020-04-15,База 1,Успешно,INV,NaT,2020-04-22,NaT
2,843083337,6769109629#16711160876,sakungurceva,2020-03-24,"Дозвон, Отложить",111,36d2f49fc49f572677416d54536a41c6,2020-04-15,База 1,Успешно,INV,NaT,2020-04-22,NaT
3,843083337,6890964670#16711160876,sakungurceva,2020-04-15,Недозвон,7,36d2f49fc49f572677416d54536a41c6,2020-04-15,База 1,Успешно,INV,NaT,2020-04-22,NaT
4,843083337,6926247810#16711160876,lapleshivenkova,2020-04-15,"Дозвон, Успешно",1329,36d2f49fc49f572677416d54536a41c6,2020-04-15,База 1,Успешно,INV,NaT,2020-04-22,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39593,933035272,7124347282#18483260522,aztrofimova,2020-05-30,Недозвон,0,9a2d6dd8ff0a11bdc7a65c8ded5d6d92,2020-05-30,База 1,Дубль,NaN,NaT,NaT,NaT
39594,933040837,7123998330#18480990905,aztrofimova,2020-05-30,Недозвон,9,3c682df2f35ed1dc634501b4b7ae25fe,2020-05-30,База 4,Отказ,NaN,NaT,NaT,NaT
39595,933040837,7124519269#18480990905,zadenis,2020-05-30,"Дозвон, Отказ",41,3c682df2f35ed1dc634501b4b7ae25fe,2020-05-30,База 4,Отказ,NaN,NaT,NaT,NaT
39596,933042718,7124237815#18482537123,npmoskalenko,2020-05-30,"Дозвон, Отказ",138,d6262e2d5840a2e68fcf028c9732606c,2020-05-30,База 1,Отказ,NaN,NaT,NaT,NaT


In [37]:
df.duplicated().value_counts()
# данные в целости
# Перехожим к следующему заданию

False    39598
dtype: int64

# Задание 2

1. На основе данных из таблицы, созданной по Заданию 1, необходимо расчитать общую воронку продаж за все время наблюдений по базам.
2. Расчитать показатели сотрудников, количество звонков , продажную конверсию ("Дозвон, Успешно"/("Дозвон, Отказ" + "Дозвон, успешно")), среднее время в звонке.
3. На основе решения П.1 визуализировать полученные данные в виде графиков/диаграмм, сформировать выводы и предложения. 

In [38]:
df.call_result_translation.unique()
#  Так выглядят все варианты резульатов звонков

array(['Дозвон, Отложить', 'Недозвон', 'Дозвон, Успешно', 'Дозвон, Отказ',
       'Дозвон, Перезвонить', 'Не было звонка'], dtype=object)

In [39]:
df.task_result_translation.unique()
# конечный результат по заданиям на дату выгрузки данных

array(['Успешно', 'Отказ', 'Недозвон', 'Дубль',
       'Неуспешно из-за другой заявки', 'Черный список', 'Не было звонка'],
      dtype=object)

## 1 Рассчитаем общую воронку продаж (3 этапа)
Для этого поставим вопросы на которые нам нужно будет ответить
- У клиентов, статус звонка которых "Дозвон, Успешно", у нас есть дальнейший путь, какие этапы проходит пользователь? 
- Какие этапы воронки у нас? Сколько ползоватлей на каждом этапе и какая конверсия?
- Зависят ли ключевые показатели от базы?

Проведем небольшой исследовательский анализ данных

In [40]:
# # для тех пользваотлеей, которые имеют статус звонка 'Дозвон, Успешно', какие значения у результата задания и дальнейшего пути?
# df[df.task_result_translation == 'Успешно'].task_id.drop_duplicates().reset_index().drop(columns = 'index')\
# .isin(\
#       df[df.call_result_translation == 'Дозвон, Успешно'].task_id.drop_duplicates().reset_index().drop(columns = 'index'))\
# .value_counts()
# # для всех потенциальных клиентов, которые имеют статус звонка успешно, результат задания успешно

In [41]:
# какие этапы проходит пользователь после продажи
df_for_funnel = df.loc[df.call_result_translation == 'Дозвон, Успешно',['task_id', 'agent_name','call_dt','base_translation','call_result_translation','task_result_translation','call_time_sec','finish_dt_task','approve_dt','reject_dt','using_dt']]
df_for_funnel

,task_id,agent_name,call_dt,base_translation,call_result_translation,task_result_translation,call_time_sec,finish_dt_task,approve_dt,reject_dt,using_dt
4,843083337,lapleshivenkova,2020-04-15,База 1,"Дозвон, Успешно",Успешно,1329,2020-04-15,NaT,2020-04-22,NaT
10,844351919,sakungurceva,2020-04-27,База 1,"Дозвон, Успешно",Успешно,785,2020-04-27,NaT,2020-05-27,NaT
37,849331398,nlkoroleva8457,2020-05-22,База 1,"Дозвон, Успешно",Успешно,942,2020-05-22,2020-05-22,NaT,NaT
53,853629907,sftarasova,2020-04-01,База 1,"Дозвон, Успешно",Успешно,902,2020-04-01,NaT,2020-04-06,NaT
81,858061897,vskrivenko,2020-04-05,База 1,"Дозвон, Успешно",Успешно,258,2020-04-05,2020-04-06,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...
39577,932925488,sftarasova,2020-05-30,База 1,"Дозвон, Успешно",Успешно,496,2020-05-30,NaT,2020-06-02,NaT
39581,932942070,zadenis,2020-05-30,База 1,"Дозвон, Успешно",Успешно,189,2020-05-30,2020-05-30,NaT,NaT
39582,932942477,sftarasova,2020-05-30,База 1,"Дозвон, Успешно",Успешно,152,2020-05-30,2020-05-30,NaT,NaT
39584,932955888,ogmixajlenko,2020-05-30,База 1,"Дозвон, Успешно",Успешно,529,2020-05-30,2020-05-30,NaT,NaT


мы можем заметить, что есть пользовали которые отказались от продукта(тут возникает вопрос, на каком этапе воронки оказывается пользователь потом, можно было бы сделать выводы, понимая, чем является продукт и какие условия по оплате, это подписка или разовая покупка).

Закрадывается предположение, что продукт - подписка на какой-либо сервис(Может быть услуги связи Тинькофф Мобайл)

А колонка approve_dt - колонка обозначающая даты, когда был предоставлен доступ к сервису, 

reject_dt - отказ от использваония сервиса(подписки), 

using_dt - очевидно, дата начала использования продукта

In [42]:
df_for_funnel_dates  = df_for_funnel[['task_id','call_dt','finish_dt_task','approve_dt','reject_dt','using_dt']]
df_for_funnel_dates

,task_id,call_dt,finish_dt_task,approve_dt,reject_dt,using_dt
4,843083337,2020-04-15,2020-04-15,NaT,2020-04-22,NaT
10,844351919,2020-04-27,2020-04-27,NaT,2020-05-27,NaT
37,849331398,2020-05-22,2020-05-22,2020-05-22,NaT,NaT
53,853629907,2020-04-01,2020-04-01,NaT,2020-04-06,NaT
81,858061897,2020-04-05,2020-04-05,2020-04-06,NaT,NaT
...,...,...,...,...,...,...
39577,932925488,2020-05-30,2020-05-30,NaT,2020-06-02,NaT
39581,932942070,2020-05-30,2020-05-30,2020-05-30,NaT,NaT
39582,932942477,2020-05-30,2020-05-30,2020-05-30,NaT,NaT
39584,932955888,2020-05-30,2020-05-30,2020-05-30,NaT,NaT


In [43]:
df_for_funnel_dates[~df_for_funnel_dates.approve_dt.isna()].count()


task_id           3724
call_dt           3724
finish_dt_task    3724
approve_dt        3724
reject_dt            0
using_dt          1953
dtype: int64

Не все пользователи, которые получили продукт, использовали его

3724 покупателя получили продукт, 1953 воспользовались

In [44]:
df_for_funnel.duplicated().value_counts() 
# дубликатов не найдено, значит нет повторных покупок


False    6011
dtype: int64

In [45]:
df_for_funnel_dates[~df_for_funnel_dates.reject_dt.isna()].count()


task_id           2264
call_dt           2264
finish_dt_task    2264
approve_dt           0
reject_dt         2264
using_dt             8
dtype: int64

2264 покупателя отказались от продукта. 
При этом 8 из них воспользовались продуктом и отказались

Дальше будем вычислять в каком промежутке это произошло 

Значит ли это, что был возврат средств? *так как у нас нет пользователей которые отказались после получения 
продукта, как будто был возврат и approve аннулировался

Тогда, видимо, дата в approve_dt означает что это настоящий покупатель  

In [62]:
# выведем датафрейм с датами, чтобы отследить более детально действия пользоватлел
df_for_funnel[['task_id','finish_dt_task','approve_dt','reject_dt','using_dt']]

,task_id,finish_dt_task,approve_dt,reject_dt,using_dt
4,843083337,2020-04-15,NaT,2020-04-22,NaT
10,844351919,2020-04-27,NaT,2020-05-27,NaT
37,849331398,2020-05-22,2020-05-22,NaT,NaT
53,853629907,2020-04-01,NaT,2020-04-06,NaT
81,858061897,2020-04-05,2020-04-06,NaT,NaT
...,...,...,...,...,...
39577,932925488,2020-05-30,NaT,2020-06-02,NaT
39581,932942070,2020-05-30,2020-05-30,NaT,NaT
39582,932942477,2020-05-30,2020-05-30,NaT,NaT
39584,932955888,2020-05-30,2020-05-30,NaT,NaT


Мы можем точно определить следующие этапы вопронки:
- База (потенциальные покупатели, которые есть в базе)
- Дозвон (все информированные люди, до которых удалось дозвониться)
- Продажа (все люди, которые купили продукт)


- Онбординг (те люди, кто получил продукт и не сделали возврат)
- Пользователи (те, кто воспользовались продуктом)

In [47]:
#  Создадим матрицу фиктивных переменных для воронки, затем высчиатем значения для решения первого пункта

In [48]:
# Каждый пользователь находится в базе, поэтому все значения столбца st1_base будут равны 1
df['st_base'] = np.zeros(df.shape[0], dtype = 'int')+1

# Все, до кого дозвонились хотя бы 1 раз, были на этапе 2, "Дозвон"
calles_users_id = df[df.call_result_translation.isin(['Дозвон, Отложить', 'Дозвон, Успешно', 'Дозвон, Отказ',
       'Дозвон, Перезвонить'])].task_id.unique()
df['st_called'] = np.where(df.task_id.isin(calles_users_id), 1, 0)

#  Для всех, кто имеет статус успешно, была совершена продажа, отразим это в датафрейме
df['st_buy'] = np.where(df.task_result_translation == 'Успешно', 1,0)

#  Онбординг
df['st_onboarding'] = np.where((df.task_result_translation == 'Успешно') & (~df.approve_dt.isna()) , 1,0)

# Пользователи
df['st_users'] = np.where((df.task_result_translation == 'Успешно') & (~df.approve_dt.isna()) &(~df.using_dt.isna()) , 1,0)


In [49]:
#  создадим датафрейм c этапами воронки
df_task_stages_marix = df.iloc[:,0].to_frame().join(df.iloc[:,-5:]).drop_duplicates()
df_funnel = df_task_stages_marix.iloc[:,-5:].sum().to_frame().reset_index().rename(columns = {0 : 'quantity', 'index' : 'stage'})
df_funnel

# Добавим конверсии между этапами
q1 = df_funnel.iloc[:,1].reset_index().drop(columns = ['index'])
q1.index = q1.index +1
df_funnel['q1'] = q1

df_funnel['q1'] = df_funnel.quantity/df_funnel['q1']
df_funnel = df_funnel.rename(columns ={'q1': 'conversion'})
df_funnel['cum_conversion'] = df_funnel.quantity/  df_funnel.iloc[0,1]
df_funnel
# df_funnel.q1

,stage,quantity,conversion,cum_conversion
0,st_base,23119,NaN,1.000000
1,st_called,18807,0.813487,0.813487
2,st_buy,6011,0.319615,0.260003
3,st_onboarding,3724,0.619531,0.161080
4,st_users,1953,0.524436,0.084476


#### Попробуем сравнить конверсии для разных баз

In [54]:
#  а какие базы у нас есть в датафрейме?
df[['task_id', 'base_translation']].drop_duplicates().base_translation.value_counts()

База 1    21980
База 4     1139
Name: base_translation, dtype: int64

In [55]:
#  создадим датафрейм c этапами воронки
# отдельно вынесем запрос
que_b1 = df.base_translation == 'База 1'
df_task_stages_marix = df[que_b1].iloc[:,0].to_frame().join(df.iloc[:,-5:]).drop_duplicates()
df_funnel = df_task_stages_marix.iloc[:,-5:].sum().to_frame().reset_index().rename(columns = {0 : 'quantity', 'index' : 'stage'})

# Добавим конверсии между этапами
q1 = df_funnel.iloc[:,1].reset_index().drop(columns = ['index'])
q1.index = q1.index +1
df_funnel['q1'] = q1

df_funnel['q1'] = df_funnel.quantity/df_funnel['q1']
df_funnel['cum_conversion'] = df_funnel.quantity/  df_funnel.iloc[0,1]
df_funnel_b1 = df_funnel.rename(columns ={'q1': 'conversion'})
df_funnel_b1


,stage,quantity,conversion,cum_conversion
0,st_base,21980,NaN,1.000000
1,st_called,17897,0.814240,0.814240
2,st_buy,5560,0.310667,0.252957
3,st_onboarding,3537,0.636151,0.160919
4,st_users,1869,0.528414,0.085032


In [56]:
#  создадим датафрейм c этапами воронки
# отдельно вынесем запрос
que_b1 = df.base_translation == 'База 4'
df_task_stages_marix = df[que_b1].iloc[:,0].to_frame().join(df.iloc[:,-5:]).drop_duplicates()
df_funnel = df_task_stages_marix.iloc[:,-5:].sum().to_frame().reset_index().rename(columns = {0 : 'quantity', 'index' : 'stage'})

# Добавим конверсии между этапами
q1 = df_funnel.iloc[:,1].reset_index().drop(columns = ['index'])
q1.index = q1.index +1
df_funnel['q1'] = q1

df_funnel['q1'] = df_funnel.quantity/df_funnel['q1']
df_funnel['cum_conversion'] = df_funnel.quantity/  df_funnel.iloc[0,1]
df_funnel_b4 = df_funnel.rename(columns ={'q1': 'conversion'})
df_funnel_b4


,stage,quantity,conversion,cum_conversion
0,st_base,1139,NaN,1.000000
1,st_called,910,0.798946,0.798946
2,st_buy,451,0.495604,0.395961
3,st_onboarding,187,0.414634,0.164179
4,st_users,84,0.449198,0.073749


In [57]:
df_comp_funnel = df_funnel_b1.merge(df_funnel_b4, on = 'stage', suffixes=('1', '4')).iloc[:,[0,1,4,2,5,3,6]]
df_comp_funnel

,stage,quantity1,quantity4,conversion1,conversion4,cum_conversion1,cum_conversion4
0,st_base,21980,1139,NaN,NaN,1.000000,1.000000
1,st_called,17897,910,0.814240,0.798946,0.814240,0.798946
2,st_buy,5560,451,0.310667,0.495604,0.252957,0.395961
3,st_onboarding,3537,187,0.636151,0.414634,0.160919,0.164179
4,st_users,1869,84,0.528414,0.449198,0.085032,0.073749


При сравнении воронок двух баз мы замечаем, что в 4 базе, которая составляет всего 1139 человек, конверсия на посленем этапе выше на 18%. На общей выборке это на 14% больше!!!

Есть смысл проверить эту гипотеу статистически

H0: Конверсии между 1 базой и 4 базой не отличаются

H1: Конверсии между 1 базой и 4 базой имеют статистически значимые различия

In [58]:

910*0.310667 # ожиданием значение конверсии последнего этапа

282.70697

In [59]:
st.chisquare([282.70697,451]) 


Power_divergenceResult(statistic=38.60198295046986, pvalue=5.19661003355646e-10)

P уровень значимости сильно меньше 0.05, сделовательно отклоняем нулевую гипотезу 

! Конверсии между 1 базой и 4 базой имеют статистически значимые различия

Поэтому, если есть возможность увеличить объем 4 базы, стоит проверить гипотезу

Но если мы посмотрим на конверсию в онбординг и пользователя, в случае 4 базы показатели не сильно отличаются

## 2 Расчитать показатели сотрудников, количество звонков , продажную конверсию ("Дозвон, Успешно"/("Дозвон, Отказ" + "Дозвон, успешно")), среднее время в звонке.


In [60]:
# посчитем простые метрики для агентов: количество звонков и их средняя длительность
agents_report = df.groupby('agent_name').agg({'call_id' : 'count', 'call_time_sec' : 'mean'}).rename(columns = {'call_id' : 'calls_count', 'call_time_sec' : 'mean_duration'})
agents_report

# посчитаем конверсию
agents_efficiency = df[df.call_result_translation == 'Дозвон, Успешно']\
.groupby('agent_name')\
.agg({'call_id': 'count'})\
.rename(columns = {'call_id' : 'sales_count'})\
.merge(df[df.call_result_translation == 'Дозвон, Отказ']\
       .groupby('agent_name')\
       .agg({'call_id': 'count'})\
       .rename(columns = {'call_id' : 'refuse_count'}),\
      on = 'agent_name')
#присоединим датафрейм со столбцом отказов
agents_efficiency['conversion'] = agents_efficiency.sales_count  /(agents_efficiency.sales_count+agents_efficiency.refuse_count)
# посчитали конверсию
agents_efficiency = agents_efficiency.drop(columns = ['sales_count','refuse_count'])
agents_efficiency

# объединим датафреймы
agents_report = agents_report.merge(agents_efficiency, on='agent_name', how = 'left').iloc[:, [0,2,1]]
agents_report

,calls_count,conversion,mean_duration
agent_name,,,
aashapovalov64931,549,0.404908,100.147541
afgluxov,263,0.516484,137.117871
amshevchenko,18,0.250000,206.555556
arxackevich,529,0.418605,140.096408
asmirov,369,0.444444,138.783198
...,...,...,...
yuapodorueva,5,NaN,40.000000
yugdulova,210,0.383721,142.895238
yuktixonova,240,0.436782,74.633333


In [61]:
agents_report.isna().value_counts()
# обратим внимание что есть менеджеры вообще без успешных звонков

calls_count  conversion  mean_duration
False        False       False            90
             True        False            16
dtype: int64

In [64]:
agents_report.sort_values('conversion', ascending=False)

,calls_count,conversion,mean_duration
agent_name,,,
takosova72025,92,0.620690,133.369565
mnafonina,132,0.617647,124.795455
isivanova94998,775,0.587537,130.100645
tiseregina,1001,0.569767,71.382617
lapleshivenkova,376,0.568493,194.638298
...,...,...,...
ogsorokoletova28872,4,NaN,189.500000
ssshamuratova,68,NaN,32.191176
tavnukova,20,NaN,21.150000
